***Remember to upload the COVID-19 dataset (`WHO-COVID-19-global.csv`) before running.***

# Installing PySpark

In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget http://archive.apache.org/dist/spark/spark-3.1.1/spark-3.1.1-bin-hadoop3.2.tgz
!cp drive/MyDrive/MMDS-data/spark-3.1.1-bin-hadoop3.2.tgz
!tar xf spark-3.1.1-bin-hadoop3.2.tgz
!pip install -q findspark

cp: cannot stat 'drive/MyDrive/MMDS-data/spark-3.1.1-bin-hadoop3.2.tgz': No such file or directory


In [2]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop3.2"

In [3]:
! echo $SPARK_HOME

/content/spark-3.1.1-bin-hadoop3.2


In [4]:
import findspark
findspark.init()

## Init

In [21]:
import csv
from pyspark import SparkConf, SparkContext

# Create a SparkConf object and a SparkContext
conf = SparkConf().setAppName("CovidDataProcessing")
sc = SparkContext.getOrCreate(conf)

# Task 1

In [22]:
try:
    # Load the CSV file into an RDD
    rdd = sc.textFile("WHO-COVID-19-global.csv")

    # Split the header row by commas to count the number of columns
    header_cols = next(csv.reader([rdd.first()]))
    num_cols = len(header_cols)

    # Remove the header row from the RDD
    rdd = rdd.filter(lambda row: row != header_cols)

    # Parse each row using the csv module and filter out rows with unexpected numbers of columns
    parsed_rdd = rdd.map(lambda row: next(csv.reader([row])))
    parsed_rdd = parsed_rdd.filter(lambda row: len(row) == num_cols)

    # Define the column indices for the numeric columns
    numeric_cols = [i for i, col in enumerate(header_cols) if col.startswith("Cases") or col.startswith("Deaths")]

    # Group the RDD by the "WHO Region" column
    grouped_rdd = parsed_rdd.groupBy(lambda row: row[1])

    # Compute the sum of each numeric column for each group
    totals = {}
    for col_idx in numeric_cols:
        col_name = header_cols[col_idx]
        total_col = grouped_rdd.mapValues(lambda rows: round(sum(float(row[col_idx].replace(',', '')) if row[col_idx].replace(',', '').replace('.', '').isnumeric() else 0 for row in rows), 2))
        totals[col_name] = total_col.collect()

    # Display the results
    for col_name, result in totals.items():
        print(f"Total {col_name}:")
        for region, total in result:
            print(f"  {region}: {total}")

except csv.Error as e:
    print(f"Error parsing CSV file: {e}")

except Exception as e:
    print(f"Unexpected error: {e}")

finally:
    # Stop the SparkContext
    sc.stop()

Total Cases - cumulative total:
  Americas: 176342137.0
  South-East Asia: 60084208.0
  Western Pacific: 85868508.0
  WHO Region: 0
  Europe: 249105808.0
  Eastern Mediterranean: 23011442.0
  Africa: 9298893.0
  Other: 764.0
Total Cases - cumulative total per 100000 population:
  Americas: 1208742.01
  South-East Asia: 65053.02
  Western Pacific: 607137.07
  WHO Region: 0
  Europe: 2038171.58
  Eastern Mediterranean: 178510.43
  Africa: 203666.51
  Other: 0.0
Total Cases - newly reported in last 7 days:
  Americas: 696441.0
  South-East Asia: 82449.0
  Western Pacific: 1806067.0
  WHO Region: 0
  Europe: 988960.0
  Eastern Mediterranean: 28094.0
  Africa: 10542.0
  Other: 0.0
Total Cases - newly reported in last 7 days per 100000 population:
  Americas: 4434.75
  South-East Asia: 61.92
  Western Pacific: 5451.1
  WHO Region: 0
  Europe: 5609.7
  Eastern Mediterranean: 360.17
  Africa: 935.76
  Other: 0.0
Total Cases - newly reported in last 24 hours:
  Americas: 29643.0
  South-East As

# Task 2

In [23]:
from pyspark.sql import SparkSession

# create a SparkSession
spark = SparkSession.builder.appName("WHO-COVID19").getOrCreate()

# read CSV data into a DataFrame
df = spark.read.csv("WHO-COVID-19-global.csv", header=True)

# select columns of interest and cast them to appropriate types
df = df.selectExpr("CAST(`Cases - cumulative total` AS INT) AS cases_total",
                   "CAST(`Cases - cumulative total per 100000 population` AS FLOAT) AS cases_total_per_100k",
                   "CAST(`Cases - newly reported in last 7 days` AS INT) AS cases_last_7_days",
                   "CAST(`Cases - newly reported in last 7 days per 100000 population` AS FLOAT) AS cases_last_7_days_per_100k",
                   "CAST(`Cases - newly reported in last 24 hours` AS INT) AS cases_last_24_hours",
                   "CAST(`Deaths - cumulative total` AS INT) AS deaths_total",
                   "CAST(`Deaths - cumulative total per 100000 population` AS FLOAT) AS deaths_total_per_100k",
                   "CAST(`Deaths - newly reported in last 7 days` AS INT) AS deaths_last_7_days",
                   "CAST(`Deaths - newly reported in last 7 days per 100000 population` AS FLOAT) AS deaths_last_7_days_per_100k",
                   "CAST(`Deaths - newly reported in last 24 hours` AS INT) AS deaths_last_24_hours",
                   "`WHO Region`")

# group by WHO Region and compute the sum of each column
grouped_df = df.groupBy("WHO Region").sum()

# write the result to disk
grouped_df.write.mode("overwrite").csv("WHO-COVID19-global/")
